In [1]:
using DualNumbers, LaTeXStrings, Plots
pyplot()

Plots.PyPlotBackend()

In [2]:
"""
    ciclosestables!(xx, f, nit, nout, cc)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0,cc)` tenga sentido. La idea es los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`. 
"""
function ciclosestables!(xx, f, nit, nout, cc)
    @assert nit > 0 && nout > 0
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    return nothing
end

"""
    diagbifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diagbifurc(f, nit, nout, crange)
    xx = Vector{Float64}(nout)
    #Lo voy a guardar en un array.
    ff = Array{Float64,2}(nout, length(crange))
    #ff = Any[]
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c)
        
        #push!(ff, xx)
        
        ff[:,ic] = xx
        
    end
    
    return ff
end

diagbifurc (generic function with 1 method)

In [3]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 1000, 256, crange); 
cc = ones(size(ff)[1])*crange';

# Esto cambia las matrices en vectores; ayuda un poco para los dibujos
#ff = reshape(ff, size(ff)[1]*size(ff)[2]);
#cc = reshape(cc, size(ff));

In [4]:
figure(figsize=(6,4))
plot(cc, ff, "b,")
plot([-1.2,-1.5,-1.5,-1.2,-1.2],[-1.5,-1.5,-0.9,-0.9,-1.5], "k-")
plot([-1,0.5],[0.0,0.0], "r-")
xlabel(L"c")
ylabel(L"x_\infty")
title("Fig. 1")

LoadError: LoadError: UndefVarError: figure not defined
while loading In[4], in expression starting on line 1

-----

In [5]:
function find_one_bif(ff, cc)

    Resta = []
    bif = 0

    for i in 1:length(ff[1,:])

        resta = abs(ff[1, i] - ff[2, i])

        if resta > 0.0001

            bif = i
            break

        end

    end

    bif, cc[1, bif]

end

find_one_bif (generic function with 1 method)

In [6]:
find_one_bif(f1, c1) #funciona! Aunque está bien chafa.

LoadError: LoadError: UndefVarError: f1 not defined
while loading In[6], in expression starting on line 1

#### voy a tratar de refinar las orbitas usando métdo de newton

Es necesario tener una función que te genera la función iterada en sí misma. Entonces...

### Metaprogramming (cuadrica y más allá...)

In [7]:
nombre(n::Int) = symbol( string("F_", n) )

nombre (generic function with 1 method)

In [8]:
function itera_funcion_identidad(n)
    
    x = "x^2 + c"

    for i in 1:n-1

        x = "($x)^2 +c - x"

    end

    ex = parse(x)
    ex_ret = :( $(nombre(n))(x, c) = $ex )
    ex_ret
end 

itera_funcion_identidad (generic function with 1 method)

Funciona con `DualNumbers` y está muy chido!

### Adecuando `compute_roots`

Lo que busco es darle de comer las orbitas de la función de arriba para que la refine

In [9]:
"""
    compute_roots(f::Function, xx, c)
    Out: roots

xx is an array, cc is the parameter, f is the funtion

"""
function compute_roots(f::Function, xx, c)
    
    roots = similar(xx)

    for j in 1:length(xx)
        
        x = Dual(xx[j], xx[j])

        # 100 iterations of Newton's method
        for i in 1:100
            x_2 = realpart(x) - realpart(f(x, c)) / dualpart(f(x, c))
            x = Dual(x_2, x_2)
        end

        roots[j] = realpart(x)
    end
    
    roots
end

compute_roots (generic function with 1 method)

In [74]:
function delete_equals(A)

    for j  in 1:length(A)
        for i  in 1:length(A)

            if i == j
                
                nothing

                elseif abs(abs(A[i]) - abs(A[j])) < 1e-5

                A[i] = NaN
                
            end
        end
    end
    
    deleteat!(A,find(isnan, A))
    
end

delete_equals (generic function with 1 method)

# Ahora integro todo lo que hice arriba al método de Luis

Pasos para acomplar las funciones:
1. `diagbifuc`: se crea una matriz con las orbitas de las iteraciones.
2. `computeroots`: refinamos las orbitas.
    - itera_funcion
    - Condiciones para aplicar la cuádrica.
    
3. Ver si poner los datos en una matriz o en un array.

--------

In [82]:
"""
    bifurcaciones(f, nit, nout, crange)
"""

function bifurcaciones(f, nit, nout, crange)
    ff = Any[]
    
    for i in eachindex(crange)
        
        xx = Vector{Float64}(nout)
        c = crange[i]
        
        ciclosestables!(xx, f, nit, nout, c)
        
        #xx = compute_roots(F_2, xx, c)
        
        delete_equals(xx)
       
        push!(ff, xx)
        
    end
    
    ff
    
end     

bifurcaciones (generic function with 1 method)

In [76]:
f(x, c) = x^2 + c

f (generic function with 1 method)

In [77]:
crange = -0.0:-0.01:-0.8

c_s = collect(crange)
fsalida= bifurcaciones(f, 1000, 200, crange)

81-element Array{Any,1}:
 [0.0]                                                                                                                                                                                                        
 [-0.009901951359278483]                                                                                                                                                                                      
 [-0.019615242270663188]                                                                                                                                                                                      
 [-0.029150262212918117]                                                                                                                                                                                      
 [-0.0385164807134504]                                                                                                                             

In [79]:
function find_bifurcation(A, crange)
    bifurca = Float64[]
    
    for i in 1:length(A)-1
        
        if length(A[i]) < length(A[i+1])
            
            bif = crange[i+1]
            
            push!(bifurca, bif)
            
        end
    end
    bifurca
    
end         

find_bifurcation (generic function with 3 methods)

In [80]:
find_bifurcation(fsalida, crange)

1-element Array{Float64,1}:
 -0.75

> Al parcer funciona, pero hay que probar para todos los valores.

funciones para calcular las bifurcaciones:
- bifurcaciones
- find_bifurcation

In [83]:
?bifurcaciones

search: bifurcaciones



```
bifurcaciones(f, nit, nout, crange)
```


In [89]:
c_range = 0.25:-0.0001:-1.3
xx1 = bifurcaciones(f, 2000, 200, c_range)
#find_bifurcation(xx1, c_range)

15501-element Array{Any,1}:
 [0.49950258007263393,0.4995127642400655,0.49952276762602793,0.4995328055804352,0.49954284842002433]
 [0.48999999999999855]                                                                              
 [0.48585786437626743]                                                                              
 [0.48267949192431026]                                                                              
 [0.47999999999999876]                                                                              
 [0.4776393202250012]                                                                               
 [0.4755051025721676]                                                                               
 [0.4735424868893533]                                                                               
 [0.47171572875253753]                                                                              
 [0.4699999999999992]                                          

In [90]:
find_bifurcation(xx1, c_range)

43-element Array{Float64,1}:
 -0.7453
 -0.746 
 -0.7464
 -0.7466
 -0.7467
 -0.7468
 -0.747 
 -0.7471
 -0.7472
 -0.7473
 -0.7474
 -0.7475
 -0.7476
  ⋮     
 -1.2486
 -1.2487
 -1.2488
 -1.2489
 -1.249 
 -1.2491
 -1.2492
 -1.2493
 -1.2494
 -1.2495
 -1.2496
 -1.2497

# Aquí abandono mi idea de poner las orbitas en arrays dentro de un array, y contar los elementos.